$$
\begin{align}
\tau(t_{n+r}) & = \frac 1k \bigg[\sum_{j=0}^r \alpha_j u(t_{n+j}) + k\sum_{j=0}^r \beta_j u'(t_{n+j})\bigg] \\ 
& = \cdots + k^{q-1} \bigg[\sum_{j=0}^r \Big(\frac{1}{q!} j^q \alpha_j - \frac{1}{(q-1)!} j^{q-1}\beta_j \Big)\bigg] u^{(q)} (t_n) + \cdots
\end{align}
$$ 

Let's name each coefficient $\{C_r\}_{r=0}^\infty = \sum_{j=0}^r \Big(\frac{1}{q!} j^q \alpha_j - \frac{1}{(q-1)!} j^{q-1}\beta_j \Big)$

To achieve the maximum efficiency possible, we want to banish the coefficients. We need to find the solution to
$$\begin{bmatrix}
1 & 0 & 1 & 0 & \cdots & 1 & 0 \\
0 & -1 & 1 & -1 & \cdots & r & 0 \\
0 & -1 & 1 & -1 & \cdots & r & 0 \\
\vdots &\vdots&\vdots&\vdots&\ddots&\vdots&\vdots \\
1 & 0 & 1 & 0 & \cdots & r & 0 \\
\end{bmatrix}
\begin{bmatrix}
\alpha_0 \\ \beta_0 \\ \alpha_1 \\ \beta_1 \\ \vdots \\ \alpha_r \\ \beta_r 
\end{bmatrix}
= 
\begin{bmatrix}
0 \\ 0 \\ 0 \\ 0 \\ \vdots \\ 0 \\ 0
\end{bmatrix}
$$


In [31]:
import numpy as np

In [32]:
def factorial(n: int) -> float:
    if n <= 1:
        return 1
    return n * factorial(n - 1)

def alpha(q: int, j: int) -> float:
    return j**q/factorial(q) if q!=0 else 1

def beta(q: int, j: int) -> float:
    return -(j)**(q-1)/factorial(q-1) if q!=0 else 0


def coefficients(r: int, q: int) -> list[float]:
    """
        r: step number
        q: order of the derivative
    """
    if r < 1:
        raise ValueError("n must be non-negative")

    c = [0] * (2*(r+1))

    # Alphas and betas
    for j in range(r+1):
        c[2*j] = alpha(q, j)
        c[2*j+1] = beta(q, j)

    return c

## Adams-Bashforth
For Adams-Bashforth method, we choose $\alpha_r = 1$, $\alpha_{r-1} = -1$, $\alpha_j = 0$ for $j<r-1$ and $\beta_{r} = 0$, then we define a system of linear equations such that 
$$\begin{bmatrix}
1 & 0 & 1 & 0 & \cdots & 1 \\
0 & -1 & 1 & -1 & \cdots & r \\
0 & -1 & 1 & -1 & \cdots & r \\
\vdots &\vdots&\vdots&\vdots&\ddots&\vdots \\
1 & 0 & 1 & 0 & \cdots & r \\
\end{bmatrix}
\begin{bmatrix}
\alpha_0 \\ \beta_0 \\ \alpha_1 \\ \beta_1 \\ \vdots \\ \alpha_r 
\end{bmatrix}
= 
\begin{bmatrix}
0 \\ 0 \\ 0 \\ 0 \\ \vdots \\ 0 \\ 0
\end{bmatrix}
$$

In [33]:

def ab_eq(order: int, q: int) -> list[float]:
    if order < 1:
        raise ValueError("r must be positive")

    Ai = [0] * order
    for j in range(order):
        Ai[j] = beta(q, j)
 
    bi = alpha(q, order-1) - alpha(q, order)
    return Ai, bi

def adams_bashforth(order: int):
    if order < 1:
        raise ValueError("r must be positive")

    A, b = [], []
    for i in range(1, order+1):
        Ai, bi = ab_eq(order, i)
        A.append(Ai)
        b.append(bi)

    A = np.array(A)
    b = np.array(b)
    x = np.linalg.solve(A, b)

    return x

In [34]:
print(f'ADAMS-BASHFORTH COEFFICIENTS')
print(f'r=1: {adams_bashforth(1)}')
print(f'r=2: {adams_bashforth(2)}')
print(f'r=3: {adams_bashforth(3)}')
print(f'r=4: {adams_bashforth(4)}')
print(f'r=4: {adams_bashforth(5)}')

ADAMS-BASHFORTH COEFFICIENTS
r=1: [1.]
r=2: [-0.5  1.5]
r=3: [ 0.41666667 -1.33333333  1.91666667]
r=4: [-0.375       1.54166667 -2.45833333  2.29166667]
r=4: [ 0.34861111 -1.76944444  3.63333333 -3.85277778  2.64027778]


We can verify that the coefficients are indeed correct.

## Adams Moulton
For Adams-Bashforth method, we choose $\alpha_r = 1$, $\alpha_{r-1} = -1$ and $\alpha_j = 0$ for $j<r-1$. We need to integrate one more coefficient into the SLE.

In [ ]:
def am_eq(order: int, q: int) -> list[float]:
    if order < 1:
        raise ValueError("r must be positive")

    Ai = [0] * (order+1)
    for j in range(order+1):
        Ai[j] = beta(q, j)
 
    bi = alpha(q, order-1) - alpha(q, order)
    return Ai, bi

def adams_moulton(order: int):
    if order < 1:
        raise ValueError("r must be positive")

    A, b = [], []
    for i in range(1, order+2):
        Ai, bi = am_eq(order, i)
        A.append(Ai)
        b.append(bi)

    A = np.array(A)
    b = np.array(b)
    x = np.linalg.solve(A, b)

    return x 

In [36]:
print(f'ADAMS-MOULTON COEFFICIENTS')
for i in range(1, 5):
    print(f'r={i}: {adams_moulton(i)}')

ADAMS-MOULTON COEFFICIENTS
r=1: (array([[-1., -1.],
       [ 0., -1.]]), array([-1. , -0.5]), array([0.5, 0.5]))
r=2: (array([[-1. , -1. , -1. ],
       [ 0. , -1. , -2. ],
       [ 0. , -0.5, -2. ]]), array([-1.        , -1.5       , -1.16666667]), array([-0.08333333,  0.66666667,  0.41666667]))
r=3: (array([[-1.        , -1.        , -1.        , -1.        ],
       [ 0.        , -1.        , -2.        , -3.        ],
       [ 0.        , -0.5       , -2.        , -4.5       ],
       [ 0.        , -0.16666667, -1.33333333, -4.5       ]]), array([-1.        , -2.5       , -3.16666667, -2.70833333]), array([ 0.04166667, -0.20833333,  0.79166667,  0.375     ]))
r=4: (array([[ -1.        ,  -1.        ,  -1.        ,  -1.        ,
         -1.        ],
       [  0.        ,  -1.        ,  -2.        ,  -3.        ,
         -4.        ],
       [  0.        ,  -0.5       ,  -2.        ,  -4.5       ,
         -8.        ],
       [  0.        ,  -0.16666667,  -1.33333333,  -4.5      

## Backward Differentation
For Backward Differentation method, we choose $\beta_j = 0$ for $j<r$ so that 
$$\sum_{j=0}^r \alpha_j U_{n+j} = k\beta_r f(U_{n+r}, t_{n+r})$$
therefore

the output is $$[\alpha_0, \ldots, \alpha_r, \beta_r]$$

In [29]:
def bd_eq(order: int, q: int) -> list[float]:
    if order < 1:
        raise ValueError("Order must be positive")

    Ai = [0] * (order+2)
    Ai[-1] = beta(q, order)  
    for j in range(order+1):
        Ai[j] = alpha(q, j)

    return Ai, 1 if q==order+1 else 0

def backward_differentation(order: int):
    if order < 1:
        raise ValueError("Order must be positive")

    A, b = [], []
    for i in range(order+2):
        Ai, bi = bd_eq(order, i)
        A.append(Ai)
        b.append(bi)

    A = np.array(A)
    b = np.array(b)
    x = np.linalg.solve(A, b)

    return A, b, x 

In [30]:
print(f'BACKWARD DIFFERENTIATION COEFFICIENTS')
for i in range(1, 5):
    print(f'r={i}: {backward_differentation(i)}')

BACKWARD DIFFERENTIATION COEFFICIENTS
r=1: (array([[ 1. ,  1. ,  0. ],
       [ 0. ,  1. , -1. ],
       [ 0. ,  0.5, -1. ]]), array([0, 0, 1]), array([ 2., -2., -2.]))
r=2: (array([[ 1.        ,  1.        ,  1.        ,  0.        ],
       [ 0.        ,  1.        ,  2.        , -1.        ],
       [ 0.        ,  0.5       ,  2.        , -2.        ],
       [ 0.        ,  0.16666667,  1.33333333, -2.        ]]), array([0, 0, 0, 1]), array([-1.5,  6. , -4.5, -3. ]))
r=3: (array([[ 1.        ,  1.        ,  1.        ,  1.        ,  0.        ],
       [ 0.        ,  1.        ,  2.        ,  3.        , -1.        ],
       [ 0.        ,  0.5       ,  2.        ,  4.5       , -3.        ],
       [ 0.        ,  0.16666667,  1.33333333,  4.5       , -4.5       ],
       [ 0.        ,  0.04166667,  0.66666667,  3.375     , -4.5       ]]), array([0, 0, 0, 0, 1]), array([ 1.33333333, -6.        , 12.        , -7.33333333, -4.        ]))
r=4: (array([[ 1.00000000e+00,  1.00000000e+00,  